# [Bacterial Colony Growth](@id Bacteries)

In this example, we are going to create a bacterial model and grow a colony using it.

 - The implementation of the force interaction dynamics is the one presented by [Volfson et al. (2008)](https://www.pnas.org/doi/abs/10.1073/pnas.0706805105)
 - We use GPU accelerated dynamics

As described in other models, it is advised that the models are constructed by parts to avoid having to find bugs in a very complex system. Hence, we will split the model in two parts:

 - Forces model
 - Growth model

## Load the packages

In [ ]:
# function repulsiveForces(
#     x,y,d,l,theta,etab,type,
#     x2,y2,d2,l2,theta2,etag,type2, Ebb, Ebg, Egg
    
# )

#     Fijx = 0.
#     Fijy = 0.
#     Wij = 0.

#     if type==type2 && type==1
#         xiAux,yiAux,xjAux,yjAux=x,y,x2,y2
#     else
#         #Function that finds the virtual spheres of contact between both rods
#         xiAux,yiAux,xjAux,yjAux = CBMMetrics.rodIntersection(x,y,l,theta,x2,y2,l2,theta2)
#     end

#     #Compute distance between virtual spheres
#     rij = sqrt((xiAux-xjAux)^2 +(yiAux-yjAux)^2)
#     if rij > 0. && rij < (d+d2)/2 #If it is smaller than a diameter compute forces
#         #Compute auxiliar
#         hAux = (d+d2)/2 - rij
#         #Compute direction
#         nijx = (xiAux-xjAux)/rij
#         nijy = (yiAux-yjAux)/rij
#         if type==0 #rod

#             #Compute the forces
#             if type2==0 #rod-rod
#                 FnAux = Ebb*d2^(1/2)*hAux^(3/2)/(etab*(l+d))
#             else #rod-gel
#                 FnAux = Ebg*d2^(1/2)*hAux^(3/2)/(etab*(l+d)) 
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #Append radial forces
#             Wij = ((xiAux-x)*Fijy - (yiAux-y)*Fijx)*12/(etab*(l+d)^3)
#         else # gel
#              #Compute the forces
#             if type2==0 #gel-rod
#                 FnAux = Ebg*d2^(1/2)*hAux^(3/2)/(etag*d)  #d2??

#             else #gel-gel
#                 FnAux = Egg*d2^(1/2)*hAux^(3/2)/(etag*d)
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #No radial forces
#         end
#     end

#     return Fijx, Fijy, Wij

# end

repulsiveForces (generic function with 1 method)

In [ ]:
# function attractiveForces(
#     x,y,d,l,theta,etab,type,
#     x2,y2,d2,l2,theta2,etag,type2, epsbb, epsbg, epsgg
    
# )

  
#     Fijx = 0.
#     Fijy = 0.
#     Wij = 0.

#     xiAux,yiAux,xjAux,yjAux=x,y,x2,y2
#     #Compute distance between central masses
#     rij = sqrt((xiAux-xjAux)^2 +(yiAux-yjAux)^2)
#     if rij > (d+d2)/2 && rij < (2*d+l) #If it is smaller than a diameter compute forces
#         #Compute direction
#         nijx = (xiAux-xjAux)/rij
#         nijy = (yiAux-yjAux)/rij
#         M=(d+d2)/2
#         D=(2*d+l)
#         if type==0 #rod

#             #Compute the forces
#             if type2==0 #rod-rod
               
#                 FnAux = epsbb*(rij/D-1)*(rij/M-1)*(rij/D)/(etab*d) 
                
#             else #rod-gel
#                 FnAux = epsbg*(rij/D-1)*(rij/M-1)*(rij/D)/(etab*d) 
          
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #Append radial forces
#             Wij = ((xiAux-x)*Fijy - (yiAux-y)*Fijx)/(etab*(l+d)^3)
#         else # gel
#             #Compute the forces
#             if type2==0 #gel-rod
#                 FnAux = epsbg*(rij/D-1)*(rij/M-1)*(rij/D)/(etag*d) 
#             else #gel-gel
#                 FnAux = epsgg*(rij/D-1)*(rij/M-1)*(rij/D)/(etag*d) 
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #No radial forces
#         end
#     end

#     return Fijx, Fijy, Wij

# end

attractiveForces (generic function with 1 method)

In [3]:
# function attractiveForces(
#     x,y,d,l,theta,etab,type,
#     x2,y2,d2,l2,theta2,etag,type2, epsbb, epsbg, epsgg
    
# )

  
#     Fijx = 0.
#     Fijy = 0.
#     Wij = 0.

#     gamma=24
#     xiAux,yiAux,xjAux,yjAux=x,y,x2,y2
#     #Compute distance between central masses
#     rij = sqrt((xiAux-xjAux)^2 +(yiAux-yjAux)^2)
#     if rij > 0. && rij < (d+d2+l2+l)*2 #If it is smaller than a diameter compute forces
#         #Compute direction
#         nijx = (xiAux-xjAux)/rij
#         nijy = (yiAux-yjAux)/rij
#         if type==0 #rod

#             #Compute the forces
#             if type2==0 #rod-rod
#                 FnAux = -gamma*epsbb*((d+d2+l2+l)/(2*(rij)))^6/rij/(etab*(l+d))
                
#             else #rod-gel
#                 FnAux = -gamma*epsbg*((d+d2+l2+l)/(2*(rij)))^6/rij/(etab*(l+d))
          
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #Append radial forces
#             Wij = ((xiAux-x)*Fijy - (yiAux-y)*Fijx)/(etab*(l+d)^3)
#         else # gel
#             #Compute the forces
#             if type2==0 #gel-rod
#                 FnAux = -gamma*epsbg*((d+d2+l2+l)/(2*(rij)))^6/rij/(etag*d) 
#             else #gel-gel
#                 FnAux = -gamma*epsgg*((d+d2+l2+l)/(2*(rij)))^6/rij/(etag*d) 
#             end
#             #Compute the interaction forces
#             Fijx = FnAux*nijx
#             Fijy = FnAux*nijy
#             #No radial forces
#         end
#     end

#     return Fijx, Fijy, Wij

# end

In [1]:
import Pkg
Pkg.activate("../")
Pkg.instantiate()


  Activating project at `c:\Users\julia\OneDrive\Escritorio\phd\Bacteria_ABM\ABM_CBM`
Precompiling packages...
   1167.0 ms  ✓ LLVMLoopInfo
   1236.5 ms  ✓ MuladdMacro
   1285.5 ms  ✓ ConcreteStructs
   1473.7 ms  ✓ SimpleUnPack
   1250.4 ms  ✓ LaTeXStrings
   1856.5 ms  ✓ RangeArrays
   1322.6 ms  ✓ IndirectArrays
   1386.8 ms  ✓ PolygonOps
   1557.8 ms  ✓ AbstractFFTs
   1638.4 ms  ✓ Calculus
   1419.3 ms  ✓ ExprTools
   1708.1 ms  ✓ FunctionWrappers
   2360.9 ms  ✓ SentinelArrays
   2017.4 ms  ✓ UnsafeAtomics
   1106.0 ms  ✓ Tricks
   1055.5 ms  ✓ InverseFunctions
   1099.0 ms  ✓ TensorCore
   1229.4 ms  ✓ WorkerUtilities
   1208.4 ms  ✓ GeoFormatTypes
   1212.9 ms  ✓ ADTypes
   1242.8 ms  ✓ Contour
   1198.8 ms  ✓ CEnum
   1229.3 ms  ✓ TriplotBase
    961.6 ms  ✓ CommonSolve
   1517.8 ms  ✓ OffsetArrays
   1478.5 ms  ✓ FillArrays
   3206.4 ms  ✓ ModernGL
   2012.0 ms  ✓ GenericSchur
   2296.8 ms  ✓ Format
    779.2 ms  ✓ CompilerSupportLibraries_jll
    780.1 ms  ✓ ManualMemory
   

In [2]:
import Pkg; Pkg.add(["Revise", "GLMakie"])

    Updating registry at `C:\Users\julia\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\julia\OneDrive\Escritorio\phd\Bacteria_ABM\ABM_CBM\Project.toml`
    Updating `C:\Users\julia\OneDrive\Escritorio\phd\Bacteria_ABM\ABM_CBM\Manifest.toml`
  [05823500] ↑ OpenLibm_jll v0.8.1+2 ⇒ v0.8.1+4
  [8e850b90] ↑ libblastrampoline_jll v5.8.0+1 ⇒ v5.11.0+0


In [3]:
using Revise
using CellBasedModels
using CUDA
using Distributions
using GLMakie
# using GeometryBasics
import GLMakie: Point3f, Cylinder, Sphere, NoShading #Can be changes to Cairo or CLMakie

## Forces model

We already use the defined forces model that is implemented in CBMModels. This is an exact copy of the rod model that you can find already predefined in `Models/rods.jl`. You can check the `Models` section of the documentation for more details.


In [11]:
rod2D = ABM(2,
    agent = Dict(
            :theta=>Float64,
            :d=>Float64,
            :l=>Float64,
            :fx=>Float64,
            :fy=>Float64,
            :W=>Float64,
            :pressure=>Float64,
            :type=>Int,
        ),    #Local Interaction Parameters

    model = Dict(
            :n=>Int,
            :Ebb=>Float64,
            :Ebg=>Float64,
            :Egg=>Float64,
            :epsbb=>Float64,
            :epsbg=>Float64,
            :epsgg=>Float64,
            :etab=>Float64,
            :etag=>Float64,
        ),        #Global parameters

    agentODE = quote

        fx = 0
        fy = 0
        W = 0
        pressure = 0
        @loopOverNeighbors i2 begin

            #Use the model provided in the documentation
            Fijx, Fijy, Wij = CBMModels.repulsiveForces(x,y,d,l,theta,etab,type, 
                                    x[i2],y[i2],d[i2],l[i2],theta[i2],etag,type[i2], Ebb, Ebg, Egg)

            #Append the repulsive forces
            fx += Fijx
            fy += Fijy
            W += Wij

            #Keep track of preassure in the media
            pressure += sqrt(Fijx^2+Fijy^2)
            #Use the model provided in the documentation
            Fijx, Fijy, Wij = CBMModels.attractiveForces(x,y,d,l,theta,etab,type, 
                                    x[i2],y[i2],d[i2],l[i2],theta[i2],etag,type[i2], epsbb, epsbg, epsgg)

            
            
            #Append the repulsive forces
            fx += Fijx
            fy += Fijy
            W += Wij
            #Keep track of preassure in the media
            pressure += sqrt(Fijx^2+Fijy^2)
            
        end

        #Equations
        dt(x) =  fx 
        dt(y) =  fy 
        dt(theta) =  W
    end,

    neighborsAlg=CBMNeighbors.CellLinked(cellEdge=4),
    platform=CPU(),
    agentAlg = CBMIntegrators.Heun()
);


### Initialize a community with two bacteries and check forces

In [12]:
com = Community(rod2D,
            N=30,
            dt=1.,
            simBox = [-10 10;-10 10.],
            );

#Natural units scales
m = 1.E-12 #ng
t = 1/(60*60) # h
d = 1.E-6 #um


com.n=30
com.Ebb=100. *m/(t^2*d)*1.E-3 #kPa
com.Ebg=100. *m/(t^2*d)*1.E-3 #kPa
com.Egg=100. *m/(t^2*d)*1.E-3 #kPa

com.epsbb=3.E8 *m*d/t^2 #Pa um^2
com.epsbg=28. *m*d/t^2 #Pa um^2
com.epsgg=18. *m*d/t^2 #Pa um^2

com.etab=500 *m/(t*d) #Pa h
com.etag=500 *m/(t*d) #Pa h


1.8000000000000003

In [6]:
com.Ebb, com.epsbb, com.etab

([1.296], [0.003887999999999999], [1.8000000000000003])

In [13]:
for i in 1:com.N
    com.theta[i] = rand() * 2π
    com.x[i] = rand()*16-8
    com.y[i] = rand()*16-8
    if i <= com.n[1]  # agentes del 0 al 4 (long rods)
        com.type[i] = 0
        com.l[i] = 3.0
        com.d[i] = 1.0
    else       # agentes del 5 al 9 (puntos o discos)
        com.type[i] = 1
        com.l[i] = 0
        com.d[i] = 0.8
    end
end

In [14]:
evolve!(com,steps=8000,saveEach=100,preallocateAgents=com.N)

In [15]:
function plotMixedAgents2D!(ax, x, y, d, l, angle, type, p; kargs...)
    for (xi, yi, li, di, ai, ti, pi) in zip(x, y, l, d, angle, type, p)


        if ti== 0  # rods
            colorval = pi
            colormap = :inferno
            meshscatter!(ax,
                [xi + li/2 * cos(ai)],
                [yi + li/2 * sin(ai)];
                marker = Sphere(Point3f(0, 0, 0), Float32(1)),
                markersize = [Point3f(di/2, di/2, di/2)],
                color = [colorval],
                colormap = colormap,
                kargs...
            )
            meshscatter!(ax,
                [xi - li/2 * cos(ai)],
                [yi - li/2 * sin(ai)];
                marker = Sphere(Point3f(0, 0, 0), Float32(1)),
                markersize = [Point3f(di/2, di/2, di/2)],
                color = [colorval],
                colormap = colormap,
                kargs...
            )
            meshscatter!(ax,
                [xi],
                [yi];
                marker = Cylinder(Point3f(-.5, 0, 0), Point3f(.5, 0, 0), Float32(1)),
                markersize = [Point3f(li, di/2, di/2)],
                rotation = [ai],
                color = [colorval],
                colormap = colormap,
                kargs...
            )
        else  # puntos
            colorval = RGBf(0, 1, 1)  # rojo puro
            meshscatter!(ax,
                [Point3f(xi, yi, 0.0)];  # ¡usar coordenadas 3D!
                marker = Sphere(Point3f(0, 0, 0), Float32(1)),
                markersize = [Point3f(di/2, di/2, di/2)],  # ¡tamaño real 3D!
                color = [colorval],
                kargs...
            )
        end

    end
    return
end


plotMixedAgents2D! (generic function with 1 method)

In [16]:
fig = Figure(size=(1500,300))

for (i,j) in enumerate([1:round(Int,length(com)/4):length(com);length(com)])
    ax = Axis3(fig[1,i],
                azimuth=3*π/2,
                elevation=π/2,
                zlabelvisible=false,
                zticklabelsvisible=false,
                zgridvisible=false,
                zspinesvisible=false
            )

    plotMixedAgents2D!(
    ax,
    com[j][:x],
    com[j][:y],
    com[j][:d],
    com[j][:l],
    com[j][:theta],
    com[j][:type],
    com[j][:pressure],
    colorrange = (0, maximum(com.pressure))
)
    xlims!(-10,10)
    ylims!(-10,10)
end

display(fig)

GLMakie.Screen(...)

## Growth model

We construct over the forces model to add bacterial growth and cell division rules.

In [109]:
rods2dGrowth = ABM(2,
    baseModelInit = [rod2D],

    agent = Dict(
                :lTarget => Float64,
            ),

    model = Dict(
                :growth=>Float64,
                :σlTarget=>Float64,
                :lMax=>Float64,
                :α=>Float64,
                :σ=>Float64,
                :p_half=>Float64,
            ),

 
    agentODE = quote
        if type == 0
            dt(l) = growth/(1+α*pressure) #linear growth with growth dependent on the pressure over the cell
        end
        
    end,

    agentRule = quote #Bound cells
        #Add division
        if l > lTarget
            ww = CBMDistributions.uniform(-.1,.1)/1000 #To add some random torque at division time
            # ww = 0 #To add some random torque at division time
            #Add a new cell
            @addAgent(
                    x=(l+d)/4*cos(theta)+x,
                    y=(l+d)/4*sin(theta)+y,
                    l=(l-d)/2,
                    W=ww,
                    lTarget = CBMDistributions.uniform(lMax-σlTarget,lMax+σlTarget), #New target length at which the bactery should divide
                    type=0)
            #Add a second cell
            @addAgent(
                    x=-(l+d)/4*cos(theta)+x,
                    y=-(l+d)/4*sin(theta)+y,
                    l=(l-d)/2,
                    lTarget = CBMDistributions.uniform(lMax-σlTarget,lMax+σlTarget), #New target length at which the bactery should divide
                    W=ww,
                    type=0)
            #Remove the old cell
            @removeAgent()
        end
        # Probability of releasing PGA with a saturating function
        # prelease = σ * pressure / (p_half + pressure)

        # if CBMDistributions.uniform(0, 1) < prelease
        #     # PGA release!
        #     # You could mark it, count it, or log it here
        #     # @info "PGA released at step $(step) by agent ID $(id)"
        #     aux=CBMDistributions.rand_pm()
        #     @addAgent(
        #             # x=((l+d)*(rand()-1/2)+aux*d/2)*cos(theta)+x,
        #             # y=((l+d)*(rand()-1/2)-aux*d/2)*sin(theta)+y,
        #             x=(CBMDistributions.uniform(-1,1)*(l+d)/2+aux*d/2)*cos(theta)+x,
        #             y=(CBMDistributions.uniform(-1,1)*(l+d)/2-aux*d/2)*sin(theta)+y,
        #             l=0,
        #             d=0.5,
        #             lTarget = CBMDistributions.uniform(lMax-σlTarget,lMax+σlTarget), #New target length at which the bactery should divide
        #             type=1)

        # end


           

    end,

    neighborsAlg=CBMNeighbors.CellLinked(cellEdge=4),
    platform=CPU(),
    agentAlg=CBMIntegrators.Heun()
);

## Grow colony from one cell

In [ ]:
com = Community(rods2dGrowth,
            N=1,
            dt=1.,
            simBox = [-100 100;-100 100.],
            );

#Natural units scales
m = 1.E-12 #ng
t = 1/(60*60) # h
d = 1.E-6 #um


com.n=30
com.Ebb=10. *m/(t^2*d)*1.E-3 #kPa
com.Ebg=100. *m/(t^2*d)*1.E-3 #kPa
com.Egg=100. *m/(t^2*d)*1.E-3 #kPa

com.epsbb=2.E7 *m*d/t^2 #Pa um^2
com.epsbg=18. *m*d/t^2 #Pa um^2
com.epsgg=18. *m*d/t^2 #Pa um^2

com.etab=500 *m/(t*d) #Pa h
com.etag=500 *m/(t*d) #Pa h




com.growth = 0.5 * t
com.lMax = 3.     
com.σlTarget = .2 
com.α = 50
com.σ = 5E-3
com.type=0

com.d = 1.        
com.l = 2.99      
com.lTarget = 3.  

com.p_half=8

com.x = 0
com.y = 0
com.theta = 0;

In [116]:
evolve!(com,
        steps=50000,saveEach=1000,
        preallocateAgents=5000,
        saveCurrentState=true,
        progressMessage=(com)->if round(Int,com.t/com.dt)%10000 == 0; println("Step: ",round(Int,com.t/com.dt), " N: ", com.N); end)

Step: 110000 N: 245
Step: 120000 N: 311
Step: 130000 N: 498
Step: 140000 N: 699
Step: 150000 N: 1019


In [117]:
# Elegimos un frame, por ejemplo el último
p = com[end][:pressure]

# Creamos la figura y el eje
fig = Figure(size = (600, 400))
ax = Axis(fig[1, 1]; xlabel = "Presión", ylabel = "Frecuencia", title = "Histograma de presión")

# Dibujamos el histograma
hist!(ax, p; bins = 30, color = :slateblue)

fig  # Esto muestra la figura en una ventana interactiva

In [118]:
fig = Figure(size=(1500,300))

for (i,j) in enumerate([1:round(Int,length(com)/4):length(com);length(com)])
    ax = Axis3(fig[1,i],
                azimuth=3*π/2,
                elevation=π/2,
                zlabelvisible=false,
                zticklabelsvisible=false,
                zgridvisible=false,
                zspinesvisible=false
            )

    plotMixedAgents2D!(
    ax,
    com[j][:x],
    com[j][:y],
    com[j][:d],
    com[j][:l],
    com[j][:theta],
    com[j][:type],
    com[j][:pressure],
    colorrange = (0, maximum(com.pressure))
    )

    xlims!(-20,20)
    ylims!(-20,20)
end

display(fig)

GLMakie.Screen(...)

In [119]:
using GLMakie

fig = Figure(size = (600, 600))
ax = Axis(fig[1, 1]; aspect = DataAspect())

# Cálculo del máximo de presión en todos los frames
max_pressure = maximum(com.pressure)

GLMakie.record(fig, "bacterias_video_pressure_atract_proves.mp4", 1:length(com); framerate = 15) do j
    # Limpiar el eje
    Makie.empty!(ax)

    # Dibujar los agentes (usando tu función personalizada)
    plotMixedAgents2D!(
        ax,
        com[j][:x],
        com[j][:y],
        com[j][:d],
        com[j][:l],
        com[j][:theta],
        com[j][:type],
        com[j][:pressure],
        colorrange = (0, max_pressure)
    )

    xlims!(ax, -50, 50)
    ylims!(ax, -50, 50)
end


"bacterias_video_pressure_atract_proves.mp4"